In [ ]:
!pip install spacy py2neo openai neo4j pandas
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://timesofindia.indiatimes.com/news"  # Replace with a real article
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
article_text = " ".join([p.text for p in soup.find_all('p')])

print(article_text[:1000])  # Preview


 'Killed not because they knew physics but ... ': Israel kills 14 Iran scientists Israel has openly acknowledged its military campaign targeting Iran's nuclear program, which involved the assassination of at least 14 senior Iranian nuclear scientists. According to Israeli officials, these individuals were directly involved in the creation and production of nuclear weapons.  Who won the war - Iran, Israel or US? The battle of Middle East dominance Amidst a week of intense exchanges between Iran and Israel, with the US acting as both aggressor and mediator, a fragile equilibrium has emerged. While missiles flew and bases were targeted, all parties are claiming victory, signaling a desire to de-escalate and avoid further conflict. Despite the heavy costs and ongoing tensions, a path towards diplomacy is cautiously being explored.  'Shared husband’s live location': Telangana man murdered by bride’s lover A 32-year-old land surveyor from Gadwal was murdered by hired killers, allegedly orche

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(article_text)

entities = [(ent.text, ent.label_) for ent in doc.ents]
print(entities)


[('Israel', 'GPE'), ('14', 'CARDINAL'), ('Iran', 'GPE'), ('Israel', 'GPE'), ('Iran', 'GPE'), ('at least 14', 'CARDINAL'), ('Iranian', 'NORP'), ('Israeli', 'NORP'), ('Iran', 'GPE'), ('Israel', 'GPE'), ('US', 'GPE'), ('Middle East', 'LOC'), ('a week', 'DATE'), ('Iran', 'GPE'), ('Israel', 'GPE'), ('US', 'GPE'), ('Shared', 'PRODUCT'), ('Telangana', 'GPE'), ('32-year-old', 'DATE'), ('Gadwal', 'GPE'), ('four', 'CARDINAL'), ('three', 'CARDINAL'), ('Nightmare', 'ORG'), ('Yashasvi', 'PERSON'), ('Gambhir', 'ORG'), ('first', 'ORDINAL'), ('Test against England', 'EVENT'), ('Yashasvi Jaiswal', 'PERSON'), ('KL Rahul', 'PERSON'), ('Rishabh Pant', 'PERSON'), ('India', 'GPE'), ('371', 'CARDINAL'), ('first', 'ORDINAL'), ('Indian', 'NORP'), ('two centuries', 'DATE'), ('New US Visa', 'ORG'), ('US', 'GPE'), ('India', 'GPE'), ('F, M', 'PRODUCT'), ('U.S.', 'GPE'), ('U.S.', 'GPE'), ('Mera', 'PERSON'), ("Rishabh Pant's", 'PERSON'), ('Jadeja', 'GPE'), ("Rishabh Pant's", 'PERSON'), ('first', 'ORDINAL'), ('Diljit

In [ ]:
triples = []
for ent in doc.ents:
    if ent.label_ in ['PERSON', 'ORG', 'GPE']:
        triples.append(("News_Article", f"has_{ent.label_}", ent.text))
print(triples)

# Example: ('News_Article', 'has_PERSON', 'Sundar Pichai')


[('News_Article', 'has_GPE', 'Israel'), ('News_Article', 'has_GPE', 'Iran'), ('News_Article', 'has_GPE', 'Israel'), ('News_Article', 'has_GPE', 'Iran'), ('News_Article', 'has_GPE', 'Iran'), ('News_Article', 'has_GPE', 'Israel'), ('News_Article', 'has_GPE', 'US'), ('News_Article', 'has_GPE', 'Iran'), ('News_Article', 'has_GPE', 'Israel'), ('News_Article', 'has_GPE', 'US'), ('News_Article', 'has_GPE', 'Telangana'), ('News_Article', 'has_GPE', 'Gadwal'), ('News_Article', 'has_ORG', 'Nightmare'), ('News_Article', 'has_PERSON', 'Yashasvi'), ('News_Article', 'has_ORG', 'Gambhir'), ('News_Article', 'has_PERSON', 'Yashasvi Jaiswal'), ('News_Article', 'has_PERSON', 'KL Rahul'), ('News_Article', 'has_PERSON', 'Rishabh Pant'), ('News_Article', 'has_GPE', 'India'), ('News_Article', 'has_ORG', 'New US Visa'), ('News_Article', 'has_GPE', 'US'), ('News_Article', 'has_GPE', 'India'), ('News_Article', 'has_GPE', 'U.S.'), ('News_Article', 'has_GPE', 'U.S.'), ('News_Article', 'has_PERSON', 'Mera'), ('New

In [ ]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "your_password"))

def insert_triple(tx, subj, pred, obj):
    tx.run("MERGE (a:Entity {name: $subj}) "
           "MERGE (b:Entity {name: $obj}) "
           "MERGE (a)-[:%s]->(b)" % pred, subj=subj, obj=obj)

with driver.session() as session:
    for s, p, o in triples:
        session.write_transaction(insert_triple, s, p, o)


/tmp/ipython-input-14-2166473212.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_triple, s, p, o)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address))
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address))
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address))
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection r

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ('127.0.0.1:7687', '[::1]:7687')):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address)